In [1]:
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal
import pprint
import datetime
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError


In [2]:
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

dynamodb = boto3.resource('dynamodb')

table = dynamodb.Table('Eve2')

In [3]:
def newTable(tableID):

    dynamodb = boto3.resource('dynamodb')


    table = dynamodb.create_table(
        TableName= tableID,
        KeySchema=[
            {
                'AttributeName': 'code',
                'KeyType': 'HASH'  #Partition key
            },
           
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'code',
                'AttributeType': 'S'
            },
            

        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 1,
            'WriteCapacityUnits': 1
        }
    )

    print("Table status:", table.table_status)

In [4]:
table = dynamodb.Table('Patients')

In [5]:
def addPatient(code,fname,lname,tn):
    response = table.put_item(
        Item={
            'code': code,
            'doses': {},
            'info': {'firstname': fname, 'lastname': lname, 'tn': tn}
                }   
    )
    return response

In [12]:
def addDose(code,drugname,timetaken):
    response = getPatient(code)
    patient_record = (response['Item'])
    
    if(drugname not in patient_record['doses']) :
        patient_record["doses"][drugname] = []
    ds1 = patient_record["doses"][drugname]
    ds1.append(timetaken)
    
    response = table.put_item(
        Item = patient_record      
    )
    return response

In [13]:
def getPatient(code):
       
    try:
        response = table.get_item(
            Key={
                'code': code
            }
        )
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        item = response['Item']
        
        #print (item['info'])
       
    return response


In [9]:
addPatient('4202','John','Doe','5555559955')

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '2',
   'content-type': 'application/x-amz-json-1.0',
   'date': 'Thu, 04 Jan 2018 01:51:46 GMT',
   'server': 'Server',
   'x-amz-crc32': '2745614147',
   'x-amzn-requestid': 'DOPQTUJBM63DSVIHLNHNQKQ977VV4KQNSO5AEMVJF66Q9ASUAAJG'},
  'HTTPStatusCode': 200,
  'RequestId': 'DOPQTUJBM63DSVIHLNHNQKQ977VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'RetryAttempts': 0}}

In [10]:
getPatient('4202')

{'Item': {'code': '4202',
  'doses': {},
  'info': {'firstname': 'John', 'lastname': 'Doe', 'tn': '5555559955'}},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '134',
   'content-type': 'application/x-amz-json-1.0',
   'date': 'Thu, 04 Jan 2018 01:51:46 GMT',
   'server': 'Server',
   'x-amz-crc32': '3947032480',
   'x-amzn-requestid': '520L4ON3GSENGOPMSHEFL3DS0JVV4KQNSO5AEMVJF66Q9ASUAAJG'},
  'HTTPStatusCode': 200,
  'RequestId': '520L4ON3GSENGOPMSHEFL3DS0JVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'RetryAttempts': 0}}

In [14]:
addDose('4202','Tylenol','2018-02-03 04:30')

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '2',
   'content-type': 'application/x-amz-json-1.0',
   'date': 'Thu, 04 Jan 2018 01:54:38 GMT',
   'server': 'Server',
   'x-amz-crc32': '2745614147',
   'x-amzn-requestid': 'RR0E4A21GISAF5OSICO6SCK4SJVV4KQNSO5AEMVJF66Q9ASUAAJG'},
  'HTTPStatusCode': 200,
  'RequestId': 'RR0E4A21GISAF5OSICO6SCK4SJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'RetryAttempts': 0}}

In [15]:
getPatient('4202')

{'Item': {'code': '4202',
  'doses': {'Tylenol': ['2018-02-03 04:30']},
  'info': {'firstname': 'John', 'lastname': 'Doe', 'tn': '5555559955'}},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '176',
   'content-type': 'application/x-amz-json-1.0',
   'date': 'Thu, 04 Jan 2018 01:54:54 GMT',
   'server': 'Server',
   'x-amz-crc32': '1976460740',
   'x-amzn-requestid': '3G0V9Q1IL3350CCPOOEINBKF6JVV4KQNSO5AEMVJF66Q9ASUAAJG'},
  'HTTPStatusCode': 200,
  'RequestId': '3G0V9Q1IL3350CCPOOEINBKF6JVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'RetryAttempts': 0}}

## play area

In [ ]:
pprint.pprint(response['Item'])

In [ ]:
patient_record = (response['Item'])

In [ ]:
pprint.pprint(patient_record)

In [ ]:
patient_record["doses"]['cocaine'] = []

In [ ]:
patient_record["doses"].keys()

In [ ]:
'cocaine' in patient_record["doses"]

In [ ]:
def drugtake(drugname,timetaken):
    if(drugname not in patient_record['doses']) :
        patient_record["doses"][drugname] = []
    ds1 = patient_record["doses"][drugname]
    ds1.append(timetaken)
    return ds1 

In [ ]:
drugtake('Heroine','2018-01-02 8:00')

In [ ]:
pprint.pprint(patient_record)

In [ ]:
expected_result={'code': '2634',
 'doses': {"cocaine": ["2018-01-01 8:00"] },
 'info': {'firstname': 'Tom', 'lastname': 'Tomson', 'tn': '5555555555'}}
expected_result == patient_record

In [ ]:
addDose('2634','addys','2018-01-03 8:00')

In [ ]:
addPatient('Patients','2634','Tom','Tomson','5555555555')

In [ ]:
response = getPatient('Patients','2634')
pprint.pprint(response)

In [ ]:
newTable('Patients')

In [ ]:
def getDoseRecords(patientID, drugName):
    table = dynamodb.Table(patientID)
    response = table.query(
        KeyConditionExpression=Key('drug').eq(drugName)
    )

    for i in response['Items']:
        print(i['drug'], ":", i['doseTime'])
        
    return response['Count'], response['Items']

In [ ]:
def getDose(drug):

    try:
        response = table.get_item(
            Key={
                'drug': drug,   
                'doseTime': doseTime,
            }
        )
    except ClientError as e:
        print(e.response['Error']['Message'])
    else:
        pprint.pprint(response)
        item = response['Item']
        print("GetItem succeeded:")
        print(json.dumps(item, indent=4, cls=DecimalEncoder))
        print(type(item))
        print (item['info'])
       
    return response


In [ ]:
cnt, items = getDoseRecords('Codine')

In [ ]:
items

In [ ]:
pprint.pprint(answer)

In [ ]:
answer["Item"]["info"]["Codine"]

In [ ]:
assert (answer["Item"]["info"]["loggeddrug"] == 'homework')

In [ ]:
newPatient('esteve')

In [ ]:
addCode("Codine", "9 o clockers")

In [ ]:
#ToDo - prompt for code and telephone #
from __future__ import print_function # Python 2/3 compatibility
import boto3
import json
import decimal
import pprint
import datetime
# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)

dynamodb = boto3.resource('dynamodb')

table = dynamodb.Table('EvE')

code = "5555" #Can this be the patient name followed by a number when the name is repeated Ex: Eve1 and Eve2 or name + code for easier categorization 

drugname = "Codeine"


# response = table.put_item(
#    Item={
#         'code': code,
#         'info': {
#             'TN': 5555555555,
#             'monthsToLabor': decimal.Decimal(6)
#         }
#     }
# )



def addCode (code, TN):
    response = table.put_item(
        Item={
            'code': code,
            'info': {
                'TN': TN,
                'loggeddrug': drugname ,
                # syntax error in the time below
                'dateOfDose':  str(datetime.datetime(2008, 11, 22, 19, 53, 42)) ,
                'timeofdose': decimal.Decimal(1400) 
                #possibly make variable for time of entry that can be overwritten
                #by time given by user if time of drug intake differs.
                
                }   
            }

    )
    return response

# response = addCode('1233',5555555556)

# print("Phone number addition succeeded:")
# print(json.dumps(response, indent=4, cls=DecimalEncoder))

try:
    response = table.get_item(
        Key={
            'code': code,   
        }
    )
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    pprint.pprint(response)
    item = response['Item']
    print("GetItem succeeded:")
    print(json.dumps(item, indent=4, cls=DecimalEncoder))
    print(type(item))
    print (item['info'])
    print (item['info']['TN'])
    # Add new Item ()



In [ ]:
response['ResponseMetadata']['HTTPStatusCode']

In [ ]:
type(response)

In [ ]:
response.keys()

In [ ]:
response = table.put_item(
        Item={
            'code': code,
            'info': {
                
                'loggeddrug': drugname ,
                # syntax error in the time below
                'dateOfDose':  str(datetime.datetime(2008, 11, 22, 19, 53, 42)) ,
                'timeofdose': decimal.Decimal(1400) 
                #possibly make variable for time of entry that can be overwritten
                #by time given by user if time of drug intake differs.
                
                }   
            }

    )

In [ ]:
response 

In [ ]:
 datetime.datetime.time(datetime.datetime.now())

In [ ]:
str datetime.datetime(datetime.datetime.now())

In [ ]:
str( datetime.datetime(datetime.datetime.now()))

In [ ]:
str(datetime.datetime.now())

In [ ]:
addCode("3332", "homework", "4")

In [ ]:
addCode()